In [1]:
import pandas as pd
import numpy as np
import string
import scipy
import seaborn as sns

from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import re

In [3]:
from wordcloud import WordCloud

In [36]:
#path = '../../../nlpdatasets//'
path='../../Code/NLP/nlp codes-a/nlpdatasets/'
inpfile = 'spam.csv'
features = ['label', 'message']

In [37]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [38]:
#sms = pd.read_csv(path+inpfile,encoding = 'unicode_escape')
sms=pd.read_csv(path+inpfile)
sms.columns = features

FileNotFoundError: [Errno 2] No such file or directory: '../../Code/NLP/nlp codes-a/nlpdatasets/spam.csv'

In [ ]:
sms.shape

In [ ]:
sms.head()

In [ ]:
sms.groupby('label').describe()

In [ ]:
sns.countplot(data=sms, x='label')

In [ ]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def generate_ngrams(words_list, n):
    ngrams_list = []
 
    for num in range(len(words_list)):
        ngram = '_'.join(words_list[num:num + n])
        if num < len(words_list) - n + 1:
            ngrams_list.append(ngram)
 
    return ngrams_list

In [ ]:
lemma = WordNetLemmatizer()

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [ ]:
sms['length'] = sms['message'].apply(len)

In [ ]:
sms.hist(column='length', by='label', bins=50)

In [ ]:
sms['tokenized_sents'] = sms['message'].map(lambda text:re.sub('[^a-zA-Z0-9]+', ' ',text)).apply(lambda x: (x.lower()).split())

In [ ]:
sms.head()

In [ ]:
sms['tokens'] = sms['tokenized_sents'].apply(lambda x: [i for i in x if i not in stop and len(i) >= 3])

In [ ]:
sms['POS_tokens'] = sms['tokens'].apply(lambda x: pos_tag(x))

sms['tokens_lemmatized'] = sms['POS_tokens'].apply(
    lambda x: [
        
            lemma.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in x

    ]
)

In [ ]:
sms['tokens_2g'] = sms['tokens_lemmatized'].apply(lambda x: generate_ngrams(x,2))

In [ ]:
sms['tokenized_clean'] = sms['tokens_lemmatized']+sms['tokens_2g']

In [ ]:
sms.head()

In [ ]:
sms.drop(['tokenized_sents','tokens_2g','POS_tokens','tokens_lemmatized','tokens','message'], axis=1, inplace=True)

sms['clean_sent'] = sms['tokenized_clean'].apply(lambda x: ' '.join(str(e) for e in x))

sms.drop(['tokenized_clean'],axis=1, inplace=True)

In [ ]:
sms.head()

In [ ]:
# convert label to a numerical variable
sms['label_num'] = sms.label.map({'ham':0, 'spam':1})

In [ ]:
sms.head()

In [ ]:
spam_df = pd.DataFrame(sms[sms.label == 'spam'].clean_sent)

In [ ]:
ham_df = pd.DataFrame(sms[sms.label == 'ham'].clean_sent)

In [ ]:
spam_words = spam_df['clean_sent'].agg(sum)

In [ ]:
ham_words = ham_df['clean_sent'].agg(sum)

In [ ]:
wc = WordCloud(width=600,height=300)

cld = wc.generate(spam_words)
plt.figure(figsize=(10,5), dpi=200, facecolor='k')
plt.imshow(cld)
plt.axis('off')
plt.tight_layout(pad=0)
plt.title('WordCloud for Spam message')
plt.show()

In [ ]:
cld = wc.generate(ham_words)
plt.figure(figsize=(10,5), dpi=200, facecolor='k')
plt.imshow(cld)
plt.axis('off')
plt.tight_layout(pad=0)
plt.title('WordCloud for Spam message')
plt.show()

In [ ]:
ham_words = sms[sms.label == 'ham'].clean_sent

In [ ]:
X = sms.drop(['label_num','label'],axis=1)
y = sms['label_num']
# split X and y into training and testing sets
# by default, it splits 75% training and 25% test
# random_state=1 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
type(X_test)

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X_train['clean_sent'].head()

In [ ]:
X_train_dtm = tfidf.fit_transform(X_train['clean_sent'])

#X_train_final = scipy.sparse.hstack((X_train_dtm, X_train.drop(['clean_sent'],axis=1).values)).tocsr()

In [ ]:
X_train_final = scipy.sparse.hstack((X_train_dtm, X_train.drop(['clean_sent'],axis=1).values)).tocsr()

In [ ]:
X_train_final

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = tfidf.transform(X_test['clean_sent'])
#X_test_final = scipy.sparse.hstack((X_test_dtm, X_test.drop(['clean_sent'],axis=1).values)).tocsr()

In [ ]:
nb = MultinomialNB()

In [ ]:
nb.fit(X_train_dtm, y_train)

In [ ]:
#make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
# calculate accuracy of class predictions with only text features
metrics.accuracy_score(y_test, y_pred_class)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred_class))

In [ ]:
# print the confusion matrix with text features
metrics.confusion_matrix(y_test, y_pred_class)

In [ ]:
# calculate predicted probabilities for X_test_dtm

# Numpy Array with 2C
# left Column: probability class 0
# right C: probability class 1
# we only need the right column 
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]

In [ ]:
#with extra features
y_pred_prob

In [ ]:
#with test features
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_prob)
auc = metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
# create plot
plt.plot(fpr, tpr, label="ROC curve, auc="+str(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.xlim([-0.02, 1])
plt.ylim([0, 1.02])
plt.legend(loc="lower right")
plt.show()